In [1]:
import polars as pl

In [23]:
df = pl.read_parquet('games/games.parquet')
print(df.columns)

team1 = 'PIT'
team2 = 'BOS'

df = df.filter(((pl.col('homeTeam') == team1) | (pl.col('awayTeam') == team1)) & ((pl.col('homeTeam') == team2) | (pl.col('awayTeam') == team2)))
df = df.with_columns(
  pl.when(pl.col('homeScore') > pl.col('awayScore')).then(pl.col('homeTeam')).otherwise(pl.col('awayTeam')).alias('winner')
)
df.group_by('winner').len()

['gameId', 'gameType', 'homeTeam', 'awayTeam', 'homeScore', 'awayScore', 'homeSog', 'awaySog', 'homeFaceoffWinningPctg', 'awayFaceoffWinningPctg', 'homePowerPlay', 'awayPowerPlay', 'homePowerPlayPctg', 'awayPowerPlayPctg', 'homePim', 'awayPim', 'homeHits', 'awayHits', 'homeBlockedShots', 'awayBlockedShots', 'homeGiveaways', 'awayGiveaways', 'homeTakeaways', 'awayTakeaways', 'season']


winner,len
str,u32
"""PIT""",23
"""BOS""",33
